In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import signal
from scipy.stats import zscore
from plotnine import *
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.stats import zscore

In [2]:
out_dir = Path("OG_data")
out_dir.mkdir(exist_ok=True)

In [3]:
df = pd.read_csv(r"C:\Users\rory\repos\ah_vids\output\OG\conditioning.csv")

In [4]:
dfj = (
    df
    .drop("trial_num", axis=1)
    .dropna()
    .assign(
        distance_z= lambda x: x.groupby("video_name")["distance_dem"].transform(lambda x: zscore(x)),
        was_jump= lambda x: x.distance_z.apply(lambda y: y > 4).astype(int),
           )
    .merge(df)
)

In [5]:
df_res = (
    dfj
    .groupby(["mouse_name"])
    .apply(lambda x: x.was_jump.mean())
    .reset_index()
    .rename(columns={0: "All_Session"})
)

In [6]:
df_res.head(2)

,mouse_name,All_Session
0,Fos Only OFL 1,0.00358
1,Fos Only OFL 1(0),0.00358


In [14]:
by_block = (
    dfj
    .groupby(["mouse_name", "block_type"])
    .apply(lambda x: x.was_jump.mean())
    .reset_index()
    .rename(columns={0: "jump_score"})
    .pivot(index="mouse_name", columns="block_type", values="jump_score")
    .rename(columns={
             "CS": "All_CS",
             "ITI": "All_ITI",
             "Post": "Two_min_Post_Experiment"})
    .reset_index()
)
by_block.index.name = ""

by_block = (
    dfj
    .loc[lambda x: x.block_type == "CS"]
    .groupby(["mouse_name"])
    .apply(lambda x: x.was_jump.mean())
    .reset_index()
    .rename(columns={0: "All_CS_excluding_US"})
    .merge(by_block)
)

In [15]:
res_ff = (
    dfj
    .loc[lambda x: x.trial_num < 5]
    .groupby(["mouse_name", "block_type"])
    .apply(lambda x: x.was_jump.mean())
    .reset_index()
    .rename(columns={0: "jump_score"})
    .pivot(index="mouse_name", columns="block_type", values="jump_score")
    .rename(columns={
             "CS": "FirstFiveTrials_CS",
             "ITI": "FirstFiveTrials_ITI"})
    .reset_index()
)

res_lf = (
    dfj
    .loc[lambda x: x.trial_num >= (x.trial_num.max() - 5)]
    .groupby(["mouse_name", "block_type"])
    .apply(lambda x: x.was_jump.mean())
    .reset_index()
    .rename(columns={0: "jump_score"})
    .pivot(index="mouse_name", columns="block_type", values="jump_score")
    .rename(columns={
             "CS": "LastFiveTrials_CS",
             "ITI": "LastFiveTrials_ITI"})
    .reset_index()
)

res_fflf = pd.merge(res_ff, res_lf)

In [16]:
res_us = (
    dfj
    .loc[lambda x: x.block_type == "CS"]
    .loc[lambda x: x.aligned >= 28 ]
    .groupby(["mouse_name"])
    .apply(lambda x: x.was_jump.mean())
    .reset_index()
    .rename(columns={0: "All_US"})
)

res_us = (
    dfj
    .loc[lambda x: x.block_type == "CS"]
    .loc[lambda x: x.aligned >= 28 ]
    .loc[lambda x: x.trial_num <= 5]
    .groupby(["mouse_name"])
    .apply(lambda x: x.was_jump.mean())
    .reset_index()
    .rename(columns={0: "FirstFiveTrials_US"})
    .merge(res_us)
)

res_us = (
    dfj
    .loc[lambda x: x.block_type == "CS"]
    .loc[lambda x: x.aligned >= 28 ]
    .loc[lambda x: x.trial_num >= (x.trial_num.max() - 5)]
    .groupby(["mouse_name"])
    .apply(lambda x: x.was_jump.mean())
    .reset_index()
    .rename(columns={0: "LastFiveTrials_US"})
    .merge(res_us)
)

In [17]:
(
    pd.merge(df_res, by_block).merge(res_us).merge(res_fflf)
    .rename(columns={"mouse_name": "Video_Name"})
    .to_excel("Dem Jump Scores.xlsx", index=False)
)

In [18]:
def save_vid(df, vid_name, colname="was_jump"):
    source_file = str(vid_dir / vid_name)
    outfile_path = str(jump_dir / f"jump_{Path(vid_name).stem}.avi")
    freezes = df.loc[lambda x: x.video_name == vid_name][colname].values
    trials = df.loc[lambda x: x.video_name == vid_name]["block_type"].values
    start_frame = df.loc[lambda x: x.video_name == vid_name]["frame"].min()
    
    save_jump_video(source_file, trials=trials, outfile_path=outfile_path, freezes=freezes, start_frame=start_frame)

In [19]:
pd.merge(df_res, by_block)

,mouse_name,All_Session,All_CS_excluding_US,Baseline,All_CS,All_ITI,Two_min_Post_Experiment
0,Fos Only OFL 1,0.003580,0.005444,0.000741,0.005444,0.001034,0.000000
1,Fos Only OFL 1(0),0.003580,0.005444,0.000926,0.005444,0.000920,0.000000
2,Fos Only OFL 1(1),0.002596,0.003926,0.000185,0.003926,0.001034,0.000000
3,Fos Only OFL 1(2),0.002282,0.003259,0.001111,0.003259,0.000575,0.000836
4,Fos Only OFL 1_2,0.002439,0.003889,0.000556,0.003889,0.000115,0.000000
5,Fos Only OFL 2(0),0.002574,0.003481,0.000000,0.003481,0.002414,0.000000
6,Fos Only OFL 2(1),0.003043,0.004074,0.003890,0.004074,0.000345,0.000557
7,Fos Only OFL 2(2),0.001947,0.003000,0.000185,0.003000,0.000575,0.000000
8,Fos Only OFL 2_1,0.004654,0.005593,0.004075,0.005593,0.002759,0.003063
9,Fos Only OFL 3,0.003916,0.004111,0.005001,0.004111,0.002644,0.003899


In [20]:
from utils import save_jump_video

vid_dir = Path(r"C:\Users\rory\repos\ah_vids\raw_videos\OG")
jump_dir = out_dir

vids = df.video_name.unique()
for i, vid in enumerate(vids):
    print(str(((i + 1) / len(vids) * 100)) + "%")
    save_vid(dfj, vid, "was_jump")

5.263157894736842%


KeyboardInterrupt: 